In [ ]:
import requests
import numpy as np
import pandas as pd

In [ ]:
# Spotify API credentials - add your own
#CLIENT_ID = "b4afc4601b004cc19e22d3bb2273b118"
#CLIENT_SECRET = "b96099f6a72f4c27bca96e9ccb6500ee"
CLIENT_ID = '92517ec8cb5a46c6b44ba12063a748f8'
CLIENT_SECRET = '18423f24021d424791386139b471d090'

import time
import requests
import pandas as pd

def get_access_token(client_id, client_secret):
    """
    Retrieve an access token from Spotify API.
    """
    url = "https://accounts.spotify.com/api/token"
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, data=data, auth=(client_id, client_secret))
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception("Failed to retrieve access token: " + response.text)

def search_song(song_name, artist_name, access_token):
    """
    Search for a song by its name and artist using Spotify API.
    """
    url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "q": f"track:{song_name} artist:{artist_name}",
        "type": "track",
        "limit": 1
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        tracks = response.json().get("tracks", {}).get("items", [])
        if tracks:
            return tracks[0]  # Return the first track
        else:
            return None
    elif response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 1))
        time.sleep(retry_after)
        return search_song(song_name, artist_name, access_token)  # Retry after sleep
    else:
        raise Exception("Failed to search for song: " + response.text)

def get_audio_features(track_id, access_token):
    """
    Get audio features for a track using Spotify API.
    """
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 1))
        time.sleep(retry_after)
        return get_audio_features(track_id, access_token)  # Retry after sleep
    else:
        raise Exception("Failed to retrieve audio features: " + response.text)

# Load the data
df = pd.read_csv('updated_streaming_data_2021.csv')

# Columns to populate
additional_columns = [
    "analysis_url", "duration_ms", "id", "time_signature",
    "track_href", "type", "uri"
]

# Get the access token
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

# Iterate through rows to populate missing columns
for index, row in df.iterrows():
    if row[additional_columns].isna().any():  # Check if any additional column is missing
        print(f"Fetching missing data for row {index + 1} of {len(df)}")
        song_data = search_song(row["track_name"], row["artist_name"], access_token)
        if song_data:  # If data is found
            for col in additional_columns:
                if col in song_data and pd.isna(row[col]):
                    df.at[index, col] = song_data[col]

# Save the updated DataFrame to a new file
updated_file_path = 'final_streaming_data_2021.csv'
df.to_csv(updated_file_path, index=False)

print(f"Missing data populated and saved to {updated_file_path}.")


Fetching missing data for row 76 of 1000
Fetching missing data for row 77 of 1000
Fetching missing data for row 78 of 1000
Fetching missing data for row 79 of 1000
Fetching missing data for row 80 of 1000
Fetching missing data for row 81 of 1000
Fetching missing data for row 82 of 1000
Fetching missing data for row 83 of 1000
Fetching missing data for row 84 of 1000
Fetching missing data for row 85 of 1000
Fetching missing data for row 86 of 1000
Fetching missing data for row 87 of 1000
Fetching missing data for row 88 of 1000
Fetching missing data for row 89 of 1000
Fetching missing data for row 90 of 1000
Fetching missing data for row 91 of 1000
Fetching missing data for row 92 of 1000
Fetching missing data for row 93 of 1000
Fetching missing data for row 94 of 1000
Fetching missing data for row 95 of 1000
Fetching missing data for row 96 of 1000
Fetching missing data for row 97 of 1000
Fetching missing data for row 98 of 1000
Fetching missing data for row 99 of 1000
Fetching missing

In [ ]:
import pandas as pd
import numpy as np

# combine and clean up data
#df = pd.read_csv('spotify_2018.csv')
#df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)
#print(df.columns)
df1 = pd.read_csv('spotify_2021.csv')
df1.drop(columns=[col for col in df1.columns if 'Unnamed' in col], inplace=True)
print(df1.columns)
df = pd.concat([df, df1])
df.tail()

Index(['acousticness', 'analysis_url', 'artist_name', 'danceability',
       'duration_ms', 'energy', 'id', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'popularity', 'speechiness', 'streams', 'tempo',
       'time_signature', 'track_href', 'track_name', 'type', 'uri', 'valence',
       'year'],
      dtype='object')


,acousticness,analysis_url,artist_name,danceability,duration_ms,energy,id,instrumentalness,key,liveness,...,streams,tempo,time_signature,track_href,track_name,type,uri,valence,year,popularity
995,0.165000,https://api.spotify.com/v1/audio-analysis/3fT6...,OneRepublic,0.665,136248.0,0.539,3fT6xqEkC1jm4mHjOyXbKw,0.00000,1.0,0.0912,...,108545632,145.828,4.0,https://api.spotify.com/v1/tracks/3fT6xqEkC1jm...,Wanted,audio_features,spotify:track:3fT6xqEkC1jm4mHjOyXbKw,0.666,2021,47.0
996,0.006070,https://api.spotify.com/v1/audio-analysis/5hNL...,Lovejoy,0.468,246125.0,0.724,5hNLVoNAD4OTWKeGFUDXwc,0.00000,0.0,0.0923,...,108539295,146.104,4.0,https://api.spotify.com/v1/tracks/5hNLVoNAD4OT...,Sex Sells,audio_features,spotify:track:5hNLVoNAD4OTWKeGFUDXwc,0.706,2021,52.0
997,0.000116,https://api.spotify.com/v1/audio-analysis/0W4N...,Electric Callboy,0.587,206133.0,0.781,0W4N0KzHKWQp2Wn1Mf6uMa,0.00002,5.0,0.1020,...,108394895,149.982,4.0,https://api.spotify.com/v1/tracks/0W4N0KzHKWQp...,We Got the Moves,audio_features,spotify:track:0W4N0KzHKWQp2Wn1Mf6uMa,0.723,2021,65.0
998,0.074700,https://api.spotify.com/v1/audio-analysis/0PrD...,Morat,0.565,211936.0,0.861,0PrD2VArqOuweEzhnpdM89,0.00000,6.0,0.0498,...,108239542,93.074,4.0,https://api.spotify.com/v1/tracks/0PrD2VArqOuw...,No Hay Más Que Hablar,audio_features,spotify:track:0PrD2VArqOuweEzhnpdM89,0.851,2021,65.0
999,0.246000,https://api.spotify.com/v1/audio-analysis/4QS7...,Yuridia,0.591,244000.0,0.799,4QS7axg5vSSSbR7CA3z5QU,0.00000,4.0,0.3340,...,108155686,145.818,4.0,https://api.spotify.com/v1/tracks/4QS7axg5vSSS...,Te Equivocaste,audio_features,spotify:track:4QS7axg5vSSSbR7CA3z5QU,0.355,2021,63.0


In [ ]:
print(df.columns)
df1 = pd.read_csv('spotify_2023.csv')
df1.drop(columns=[col for col in df1.columns if 'Unnamed' in col], inplace=True)
df1.drop(columns=['Artist and Title', 'Daily Streams'], inplace=True)
df1.rename(columns={'Streams':'streams', 'song_name':'track_name'}, inplace=True)
df1['year'] = 2023
df1.head()
df = pd.concat([df, df1])
df['year'].unique()
df.tail(20)

Index(['acousticness', 'analysis_url', 'artist_name', 'danceability',
       'duration_ms', 'energy', 'id', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'streams', 'tempo', 'time_signature',
       'track_href', 'track_name', 'type', 'uri', 'valence', 'year',
       'popularity'],
      dtype='object')


,acousticness,analysis_url,artist_name,danceability,duration_ms,energy,id,instrumentalness,key,liveness,...,streams,tempo,time_signature,track_href,track_name,type,uri,valence,year,popularity
865,0.181000,https://api.spotify.com/v1/audio-analysis/7ggn...,lauren spencer smith,0.359,174050.0,0.449,7ggnnK9KC8ZKhahPdnY0c3,0.000000,5.0,0.0974,...,"102,282,134",174.683,3.0,https://api.spotify.com/v1/tracks/7ggnnK9KC8ZK...,that part,audio_features,spotify:track:7ggnnK9KC8ZKhahPdnY0c3,0.371,2023,NaN
866,0.174000,https://api.spotify.com/v1/audio-analysis/5BTi...,julión álvarez y su norteño banda,0.719,190707.0,0.474,5BTibB0lM8fExn033QebZJ,0.000000,7.0,0.1180,...,"102,243,556",129.952,4.0,https://api.spotify.com/v1/tracks/5BTibB0lM8fE...,casi completa,audio_features,spotify:track:5BTibB0lM8fExn033QebZJ,0.772,2023,NaN
867,0.391000,https://api.spotify.com/v1/audio-analysis/3d1i...,veigh,0.899,161344.0,0.544,3d1im1pw3WLuEty2k9XzTr,0.000000,0.0,0.1380,...,"102,014,580",119.022,4.0,https://api.spotify.com/v1/tracks/3d1im1pw3WLu...,clickbait,audio_features,spotify:track:3d1im1pw3WLuEty2k9XzTr,0.753,2023,NaN
868,0.482000,https://api.spotify.com/v1/audio-analysis/2toC...,nina chuba,0.820,192836.0,0.679,2toCf2FDbTIdWriPI6kcsY,0.000000,8.0,0.1050,...,"101,858,808",145.026,4.0,https://api.spotify.com/v1/tracks/2toCf2FDbTId...,ich hass dich,audio_features,spotify:track:2toCf2FDbTIdWriPI6kcsY,0.516,2023,NaN
869,0.190000,https://api.spotify.com/v1/audio-analysis/618O...,viviz,0.753,195520.0,0.579,618OKP1lBkNJL8uZdNSvQE,0.000000,9.0,0.0931,...,"101,650,516",114.961,4.0,https://api.spotify.com/v1/tracks/618OKP1lBkNJ...,maniac,audio_features,spotify:track:618OKP1lBkNJL8uZdNSvQE,0.818,2023,NaN
870,0.135000,https://api.spotify.com/v1/audio-analysis/3Eax...,rosalía,0.766,176590.0,0.684,3Eax1yebRxj6LSYpxE9Yd3,0.019900,9.0,0.0832,...,"101,616,759",102.045,4.0,https://api.spotify.com/v1/tracks/3Eax1yebRxj6...,vampiros,audio_features,spotify:track:3Eax1yebRxj6LSYpxE9Yd3,0.474,2023,NaN
871,0.001780,https://api.spotify.com/v1/audio-analysis/62ZA...,kordhell,0.598,126857.0,0.971,62ZAFyVwbc9f2M8qCW1qJr,0.000000,2.0,0.1000,...,"101,607,126",140.200,4.0,https://api.spotify.com/v1/tracks/62ZAFyVwbc9f...,murder plot,audio_features,spotify:track:62ZAFyVwbc9f2M8qCW1qJr,0.761,2023,NaN
872,0.024200,https://api.spotify.com/v1/audio-analysis/4kf3...,jonas brothers,0.399,249693.0,0.767,4kf33kU0KAlkNBK9pNbYNU,0.000000,2.0,0.3060,...,"101,359,379",135.943,4.0,https://api.spotify.com/v1/tracks/4kf33kU0KAlk...,when you look me in the eyes,audio_features,spotify:track:4kf33kU0KAlkNBK9pNbYNU,0.507,2023,NaN
873,0.000156,https://api.spotify.com/v1/audio-analysis/6vb4...,doja cat,0.831,195573.0,0.362,6vb4kCEIM7zrwuHlxFbfeB,0.016200,1.0,0.1680,...,"101,292,638",139.941,4.0,https://api.spotify.com/v1/tracks/6vb4kCEIM7zr...,demons,audio_features,spotify:track:6vb4kCEIM7zrwuHlxFbfeB,0.313,2023,NaN
874,0.378000,https://api.spotify.com/v1/audio-analysis/6Hex...,metro boomin,0.596,319493.0,0.630,6HexNTb392JS071DoTGo0y,0.004070,6.0,0.1900,...,"101,289,766",81.408,4.0,https://api.spotify.com/v1/tracks/6HexNTb392JS...,hummingbird (metro boomin & james blake),audio_features,spotify:track:6HexNTb392JS071DoTGo0y,0.240,2023,NaN


In [ ]:
df.to_csv('spotify_complete.csv')

In [ ]:
# Get genre
import time
import requests
import pandas as pd

# Spotify API credentials
CLIENT_ID = "b4afc4601b004cc19e22d3bb2273b118"
CLIENT_SECRET = "b96099f6a72f4c27bca96e9ccb6500ee"
#CLIENT_ID = '92517ec8cb5a46c6b44ba12063a748f8'
#CLIENT_SECRET = '18423f24021d424791386139b471d090'

def get_access_token(client_id, client_secret):
    """
    Retrieve an access token from Spotify API.
    """
    url = "https://accounts.spotify.com/api/token"
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, data=data, auth=(client_id, client_secret))
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception("Failed to retrieve access token: " + response.text)

def get_track_artist_id(track_id, access_token):
    """
    Retrieve the artist ID for a given track using Spotify API.
    """
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        artists = response.json().get("artists", [])
        if artists:
            return artists[0].get("id")  # Return the first artist's ID
    elif response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 1))
        time.sleep(retry_after)
        return get_track_artist_id(track_id, access_token)  # Retry after sleep
    else:
        print(f"Failed to retrieve artist ID for track {track_id}: {response.text}")
        return None

def get_artist_genres(artist_id, access_token):
    """
    Retrieve the genres associated with an artist using Spotify API.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get("genres", [])
    elif response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 1))
        time.sleep(retry_after)
        return get_artist_genres(artist_id, access_token)  # Retry after sleep
    else:
        print(f"Failed to retrieve genres for artist {artist_id}: {response.text}")
        return None

# Load the updated data with tracks
file_path = 'spotify_2018.csv'
df = pd.read_csv(file_path)

# Ensure we have access_token
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

# Add new columns for artist ID and genres
df["artist_id"] = None
df["genres"] = None

# Iterate through each row to populate artist ID and genres
for index, row in df.iterrows():
    track_id = row.get("id")  # Use the track ID
    if pd.isna(track_id):
        print(f"Missing track ID for row {index + 1}")
        continue

    # Fetch artist ID if missing
    if pd.isna(row["artist_id"]):
        artist_id = get_track_artist_id(track_id, access_token)
        df.at[index, "artist_id"] = artist_id
    else:
        artist_id = row["artist_id"]

    # Fetch genres if artist ID is available
    if artist_id and pd.isna(row["genres"]):
        genres = get_artist_genres(artist_id, access_token)
        if genres:
            df.at[index, "genres"] = ", ".join(genres)  # Store genres as a comma-separated string

# Save the updated DataFrame
updated_genre_file_path = 'spotify_2018_updated.csv'
df.to_csv(updated_genre_file_path, index=False)

print(f"Genres added and saved to {updated_genre_file_path}.")

Failed to retrieve genres for artist 1W7FNibLa0O0b572tB2w7t: {"error": {"status": 500, "message": "com.spotify.hermes.service.RequestTimeoutException: Request timed out: 06274bd0d35000-004d1c-f935-9636-f26262f4. Host: 0a5a178c2a142a000005013c00000f1a0000000c.ip.gue1.spotify.net.:5700" } }
Missing track ID for row 592
Missing track ID for row 668
Genres added and saved to spotify_2018_updated.csv.


In [70]:
import pandas as pd
import numpy as np

df = pd.read_csv('spotify_2018_with_genre.csv')
df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
df['genres'] = df['genres'].apply(lambda x: np.nan if isinstance(x, float) else x.split(', '))
df = df.dropna()
unique_genres = np.unique(sum(df['genres'], []))

In [37]:
df.columns

Index(['acousticness', 'analysis_url', 'artist_name', 'danceability',
       'duration_ms', 'energy', 'id', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'streams', 'tempo', 'time_signature',
       'track_href', 'track_name', 'type', 'uri', 'valence', 'year',
       'popularity', 'artist_id', 'genres'],
      dtype='object')

In [71]:
import pandas as pd
import statsmodels.api as sm

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

df_exploded = df.explode('genres', ignore_index=True)
len(df_exploded['genres'].unique())


393

ValueError: y contains previously unseen labels: [23, 31, 55, 72, 76, 83, 93, 95, 108, 125, 146, 161, 173, 199, 202, 229, 241, 247, 249, 259, 260, 261, 278, 303, 327, 360, 363, 367, 368, 369, 375, 382, 384, 385, 392]

In [65]:
# fitting model
from xgboost import XGBClassifier

model = XGBClassifier(objective='multi:softmax', num_class=393)
model.fit(X_train, y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 

In [47]:
# evaluation
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.038525963149078725
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         7
           9       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         3
          21       0.00      0.00      0.00         3
          22       0.00      0.00      0.00         1
          23       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         4
          25       0.00      0.00     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_